In [1]:
import spotipy
import spotipy.util as util
import requests
import time #audio analysis
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import csv

In [2]:
cid = '049ade7215e54c63a2b628f3784dc407'
secret = 'b86bedbebf53486b970e92b2cb7b612c'
redirect_uri = 'http://google.com/'
username = 'xxx'

In [3]:
scope = 'user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [19]:
#Files
playlist_file = 'Data/playlist_datafile.csv'

#Master List
all_playlists =[]


In [20]:
def get_playlist_tracks(nexturl):
    #Get playlists
    try:
        resp = requests.get(url=nexturl,
                            headers={'Authorization': 'Bearer ' + token})
        resp.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
    
    #Generate json
    response = resp.json()
    
        
    #Get limit to set range for loop
    response_limit = response['limit']-1
    
    for x in range(response_limit):
        try:
            track_uri = (response['items'][x]['track']['uri']).split(':')
            track_uri = track_uri[2]
            track_title = response['items'][x]['track']['name']
            artist_uri = (response['items'][x]['track']['artists'][0]['uri']).split(':')
            artist_uri = artist_uri[2]
            artist_name = response['items'][x]['track']['artists'][0]['name']
            
            all_playlists.append({'track_uri':track_uri,'track_title':track_title,'artist_uri':artist_uri
                                  ,'artist_name':artist_name})
        
        except IndexError as error:
            continue #Natural part of the process. The length of response is sometimes less than limit
            
    try:    
        if (nexturl is not None):#is not none
            get_playlist_tracks(response['next'])
        else:
            return
    except:
        return

    

In [21]:
#Read file for playlist ID
with open(playlist_file) as csvfile:
    playlist_reader = csv.reader(csvfile,delimiter=':')
    next(playlist_reader)
    head = [next(playlist_reader) for x in range(1)] #for limiting amount of rows read
    for row in head:
        playlist_id_lookup = row[2].strip()
        get_playlist_tracks('https://api.spotify.com/v1/playlists/{p}/tracks'.format(p=playlist_id_lookup))
    

In [22]:
'''
for index,x in enumerate(all_playlists):
    print(index,x)
    for key,value in x.items():
        if value is None:
            print(key,value)
'''


'\nfor index,x in enumerate(all_playlists):\n    print(index,x)\n    for key,value in x.items():\n        if value is None:\n            print(key,value)\n'

In [23]:
a = pd.DataFrame(all_playlists)
a

,track_uri,track_title,artist_uri,artist_name
0,6wJYhPfqk3KGhHRG76WzOh,Blueberry Faygo,5zctI4wO9XSKS8XwcnqEHk,Lil Mosey
1,3Q6F8RByyhRTJpRtZLY3cg,WHATS POPPIN,2LIk90788K0zvyj2JJVwkJ,Jack Harlow
2,3kW4z2pIj5VY5fLjBrMRL5,P2,4O15NlyKLIASxsJ0PrXPfz,Lil Uzi Vert
3,127QTOFJsJQp5LbJbu3A1y,Toosie Slide,3TVXtAsR1Inumwj472S9r4,Drake
4,4HDCLYli2SUdkq9OjmvhSD,BELIEVE IT,2HPaUgqeutzr3jx5a9WyDV,PARTYNEXTDOOR
5,07KXEDMj78x68D884wgVEm,High Fashion (feat. Mustard),757aE44tKEUQEqRuT6GnEB,Roddy Ricch
6,2o1dvKh58G11C51Y9EgBs3,The War (feat. Young Thug),6C1ohJrd5VydigQtaGy5Wa,Joyner Lucas
7,1lGHa2pwYzxQHFBUynhLtO,FIND MY WAY,4r63FhuTkUYltbVAg5TQnk,DaBaby
8,0JjM9bKm4wrwohMslcm892,Still Be Friends (feat. Tory Lanez & Tyga),02kJSzxNuaWGqwubyUba0Z,G-Eazy
9,2w1rq80FIrbxeJy9WYURxw,Follow,790FomKkXshlbRYZFtlgla,KAROL G


In [115]:
all_playlists

[{'track_uri': 'spotify:track:127QTOFJsJQp5LbJbu3A1y',
  'track_title': 'Toosie Slide',
  'artist_uri': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'},
    'href': 'https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4',
    'id': '3TVXtAsR1Inumwj472S9r4',
    'name': 'Drake',
    'type': 'artist',
    'uri': 'spotify:artist:3TVXtAsR1Inumwj472S9r4'}],
  'artist_name': 'spotify:album:3xIwVbGJuAcovYIhzbLO3J'},
 {'track_uri': 'spotify:track:364dI1bYnvamSnBJ8JcNzN',
  'track_title': 'Intentions',
  'artist_uri': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1uNFoZAHBGtllmzznpCI3s'},
    'href': 'https://api.spotify.com/v1/artists/1uNFoZAHBGtllmzznpCI3s',
    'id': '1uNFoZAHBGtllmzznpCI3s',
    'name': 'Justin Bieber',
    'type': 'artist',
    'uri': 'spotify:artist:1uNFoZAHBGtllmzznpCI3s'}],
  'artist_name': 'spotify:album:55zg331p7m1EFA4uRggkwt'},
 {'track_uri': 'spotify:track:55CHeLEfn5iJ0IIkgaa4si',
  'track_title': 'Savag